In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.io_utils import read_data, clean_data

df = read_data("data/ecommerce_sales.csv")

df_clean = clean_data(df)

print("Here is a sample of our data:")
df_clean.sample(3)

Here is a sample of our data:


,order_id,date,city,category,price,units,revenue
1613,101613,2024-06-02,Malmö,Toys,137.77,3,413.31
2488,102488,2024-06-27,Stockholm,Electronics,586.85,3,1760.55
1872,101872,2024-03-13,Göteborg,Electronics,854.14,3,2562.42


## Total revenue and units sold

In [9]:
from src.metrics import total_revenue, total_units

print()
print(f"In 2024 we sold a total of {total_units(df_clean)} units,\n"
      f"for a total revenue of {total_revenue(df_clean)} sek.")


In 2024 we sold a total of 7463 units,
for a total revenue of 3502767.23 sek.
